In [1]:
import glob
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from keras import utils, losses, optimizers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D

SEED = 2017

Using TensorFlow backend.


In [ ]:
# Specify data directory and extract all file names for both classes
DATA_DIR = 'Data/PetImages/'
cats = glob.glob(DATA_DIR + "Cat/*.jpg")
dogs = glob.glob(DATA_DIR + "Dog/*.jpg")

print('#Cats: {}, #Dogs: {}'.format(len(cats), len(dogs)))
# #Cats: 12500, #Dogs: 12500

In [ ]:
dogs_train, dogs_val, cats_train, cats_val = train_test_split(dogs, cats, test_size=0.2, random_state=SEED)

In [ ]:
def batchgen(cats, dogs, batch_size, img_size=50):
    # Create empty numpy arrays
    batch_images = np.zeros((batch_size, img_size, img_size, 3))
    batch_label = np.zeros(batch_size)

    # Custom batch generator 
    while 1:
        n = 0
        while n < batch_size:
            # Randomly pick a dog or cat image
            if np.random.randint(2) == 1:
                i = np.random.randint(len(dogs))
                img = cv2.imread(dogs[i])
                if img is None:
                    break
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # The images have different dimensions, we resize all to 100x100
                img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_AREA)
                y = 1

            else:
                i = np.random.randint(len(cats))
                img = cv2.imread(cats[i])
                if img is None:
                    break
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_AREA)
                y = 0
        
            batch_images[n] = img
            batch_label[n] = y
            n+=1
        yield batch_images, batch_label

In [ ]:
def create_model(init_type='xavier', img_size=100):
    # Define architecture
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.) - 0.5, input_shape=(img_size, img_size, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=init_type))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=init_type))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=init_type))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=init_type))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
 
    sgd = optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])
 
    return model

In [ ]:
models = []
for init_type in ['random_uniform', 'glorot_normal', 'glorot_uniform', 'lecun_uniform', 'he_uniform']:
    model = create_model(init_type, img_size=50)
    models.append(dict({'setting': '{}'.format(init_type), 
            'model': model
            }))

In [ ]:
callbacks = [EarlyStopping(monitor='val_binary_accuracy', patience=3)]

In [ ]:
batch_size = 512
n_epochs = 500
steps_per_epoch = 100
validation_steps = round((len(dogs_val)+len(cats_val))/batch_size)

train_generator = batchgen(dogs_train, cats_train, batch_size)
val_generator = batchgen(dogs_val, cats_val, batch_size)

history = []
for i in range(len(models)):
    print(models[i])
    history.append(
        models[i]['model'].
        fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=n_epochs, validation_data=val_generator, validation_steps=validation_steps, callbacks=callbacks)
    )

In [ ]:
for i in range(len(models)):
    plt.plot(range(len(history[i].history['val_binary_accuracy'])), history[i].history['val_binary_accuracy'], label=models[i]['setting'])
    print('Max accuracy model {}: {}'.format(models[i]['setting'], max(history[i].history['val_binary_accuracy'])))
plt.title('Accuracy on the validation set')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()